In [1]:
using CSV
using DataFrames
using Dates
using Statistics
using LinearAlgebra
using JuMP
using Gurobi

data_dir = "./data"

# ─── HELPERS ───────────────────────────────────────────────────────────────────

function compute_returns(prices::Vector{Float64})
    diff(prices) ./ prices[1:end-1]
end

# ─── LOAD, COMPUTE RETURNS & LIQUIDITY ────────────────────────────────────────

csv_files    = filter(f->endswith(lowercase(f), ".csv"), readdir(data_dir; join=true))
n            = length(csv_files)
dfs_returns  = Vector{DataFrame}(undef, n)
liquidity    = Vector{Float64}(undef, n)
names        = Vector{String}(undef, n)

for (i, file) in enumerate(csv_files)
    # extract company name from filename, e.g. "boeing.csv" → "boeing"
    names[i] = split(basename(file), ".")[1]

    df = CSV.read(file, DataFrame)
    df.Date = Date.(df.Date, dateformat"u d, Y")
    sort!(df, :Date)

    rs = compute_returns(df.Adj_Close)
    vs = df.Volume[2:end]

    sub = df[2:end, [:Date]]
    sub.:return = rs

    dfs_returns[i] = sub
    liquidity[i]   = mean(vs)
end

# rename return columns r1…rn
for i in 1:n
    rename!(dfs_returns[i], :return => Symbol("r$i"))
end
# ─── BUILD `rets` DATAFRAME WITH COLUMNS Date, r1…rn ───────────────────────────

# perform an inner‐join of all per‐stock return‐tables on :Date
rets = reduce((df1, df2) -> innerjoin(df1, df2, on = :Date), dfs_returns)
sort!(rets, :Date)


# check
println("rets has $(nrow(rets)) dates × $(ncol(rets)-1) stocks of returns:")
first(rets, 52) |> display


# join on Date
rets = reduce((a,b)->innerjoin(a,b,on=:Date), dfs_returns)

# ─── COMPUTE μ AND Σ ──────────────────────────────────────────────────────────

# μ     = [ mean(rets[!, Symbol("r$i")]) for i in 1:n ]
R     = Matrix(rets[:, Not(:Date)])
Σ     = Symmetric(cov(R, dims=1))

# println("μ = ", μ)
println("Σ = "); display(Σ)

# ─── LIQUIDITY VECTOR ─────────────────────────────────────────────────────────

println("Average weekly volumes (liquidity) = ", liquidity)


rets has 52 dates × 10 stocks of returns:


Row,Date,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2024-05-13,-0.0148282,0.00212864,0.038641,0.0360764,-0.0035819,0.0285445,0.0303202,0.0315318,0.0131924,0.0727425
2,2024-05-20,-0.021386,-0.0190754,0.000581303,-0.0563936,-0.0163399,-0.0139636,-0.019935,-0.0495895,0.0255138,0.0112237
3,2024-05-27,-0.0238451,0.00764266,0.011989,0.0177057,0.0149502,-0.0101057,0.00958401,0.00603933,-0.0349335,0.00585787
4,2024-06-03,0.0445477,-0.0305495,0.0241115,0.0711109,0.0155483,0.00248506,-0.0132297,0.00279213,0.0210065,0.00183908
5,2024-06-10,-0.0034726,-0.0337289,0.0792437,-0.068177,-0.0212732,-0.0185582,-0.0308566,-0.0104413,0.0441623,0.0172862
6,2024-06-17,0.0295111,0.0246952,-0.0235622,-0.00400519,0.0113618,0.00832821,0.0129891,0.0220174,0.0162951,0.0132331
7,2024-06-24,0.0220542,0.00509219,0.0150878,0.0308677,0.014002,0.00476158,0.0303883,-0.0174134,-0.00630619,-0.00296824
8,2024-07-01,0.0349288,0.0176013,0.074651,0.0154937,0.00176622,0.0274683,0.0124949,0.00217148,0.0461113,0.0345341
9,2024-07-08,-0.02755,0.0157517,0.0185743,-0.0136342,-0.000961693,0.0325486,0.00624532,0.0232753,-0.0299451,-0.0115108


Σ = 


10×10 Symmetric{Float64, Matrix{Float64}}:
  0.00186014   0.00110124   0.000760099  …   0.000999223  0.000699394
  0.00110124   0.00142203   0.000519402      0.000648364  0.000781654
  0.000760099  0.000519402  0.00159101       0.000505046  0.00061901
  0.0013958    0.00143332   0.00103061       0.000826106  0.000938006
 -0.000123485  7.45066e-5   0.000270781     -0.00010189   7.08366e-5
  0.00107309   0.00138979   0.000648584  …   0.000598146  0.000649049
  0.00099663   0.00129395   0.000644582      0.000562353  0.000767245
 -0.000118589  0.000149548  0.000206082     -0.000152874  2.27392e-5
  0.000999223  0.000648364  0.000505046      0.00124881   0.000548978
  0.000699394  0.000781654  0.00061901       0.000548978  0.00124903

Average weekly volumes (liquidity) = [1.9773766923076922e8, 1.3400782692307692e7, 2.6582938076923078e8, 4.305632115384615e7, 7.354774807692307e7, 1.126198076923077e7, 4.575230192307692e7, 3.741719423076923e7, 1.0168295384615384e8, 8.472379807692307e7]


In [2]:
using CSV, DataFrames, Dates, Statistics, LinearAlgebra
using JuMP, Gurobi

# === PARAMETERS ===
λ       = 1e-4                   # regularization strength
max_lag = 5
t_start, t_end = 6, 48          # inclusive indices in the returns series

# --- assume you already have `rets` DataFrame with columns Date, r1…rn
R = Matrix(rets[:, Not(:Date)])  # size #obs × n
T, n = size(R)
@assert T ≥ t_end "Not enough rows: T=$(T) < t_end=$(t_end)"

# grab the stock‐column symbols unambiguously
col_syms = DataFrames.names(rets, Not(:Date))   # Vector{Symbol}

# build empty coef_df
coef_df = DataFrame(stock=String[])
for k in 0:max_lag
    coef_df[!, Symbol("b$k")] = Float64[]
end

println(R[:,10])

# 2) Loop over each stock by index and name
for (i, sym) in enumerate(col_syms)
    # extract the i'th column of returns
    r = R[:, i]

    # build the regression matrices
    N = t_end - max_lag
    X = ones(N, max_lag+1)
    y = zeros(N)
    for row in 1:N
        t = max_lag + row
        y[row] = r[t]
        X[row, 2:end] = @view r[(t-1):-1:(t-max_lag)]
    end

    # solve the ridge‐regression via a QP
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, b[1:max_lag+1])

    # build the penalty diagonal: no penalty on b[1], penalty λ on b[2]…b[p]
    D = Diagonal(vcat(0.0, ones(max_lag)))    # size (max_lag+1)×(max_lag+1)

    # then form Q using +, not .+
    Q = 2 * (X'X + λ * D)

    # rest is the same
    c = -2 * (X' * y)
    @objective(model, Min, 0.5 * b' * Q * b + c' * b)
    optimize!(model)
    b_opt = value.(b)

    # store results: vcat(stock_name, b_opt) is a Vector{Any}, push! treats it as a row
    push!(coef_df, vcat(names[i], b_opt))
end

# println("Estimated AR(5) coefficients:")
# show(coef_df, allcols=true)

# estimate μ
# Prepare a container for μ
μ = Float64[]

for (i, sym) in enumerate(col_syms)
    # 1) pull out the fitted coefficients for this stock:
    #    coef_df has columns: :stock, :b0, :b1, …, :b5
    b_vec = collect(coef_df[i, 2:end])   # Vector{Float64} of length 6

    # 2) build the lagged‐returns vector [1, r(t_end-1), …, r(t_end-5)]
    #    note R is #obs×n, so R[row, i] is return of stock i at time ‘row’
    lags = [1.0; R[(t_end-1):-1:(t_end-max_lag), i]]

    # 3) dot‐product gives the forecast μ_i
    push!(μ, dot(b_vec, lags))
end

# 4) attach μ back to coef_df
coef_df[!, :μ] = μ


# pick just the name and forecast columns
ret_df = select(coef_df, :stock, :μ)

println("Expected one-step-ahead returns by stock:")
display(coef_df)


# println("Estimated AR(5) coefficients and next‐period expected returns:")
# show(coef_df, allcols=true)


[0.0727424749163881, 0.011223694466095072, 0.005857869585324424, 0.0018390804597701847, 0.017286216919075958, 0.013233082706766848, -0.0029682398337784006, 0.03453408752604931, -0.011510791366906433, 0.021834061135371178, -0.013675213675213788, -0.018919699595609303, -0.007507728544089578, 0.08098487095817256, 0.03361690450054889, 0.02017788397716708, -0.00767729342875718, 0.05166535536323102, -0.019077306733167095, 0.009152154569721608, 0.014485451568207654, -0.010429600198659093, 0.015181932245922129, 0.01470769991348409, -0.003897685749086397, 0.03215945218879916, -0.006871223788650614, 0.07348204699988067, 0.022780308923213817, 0.03465884398087785, -0.015226294235010006, -0.019087225421198617, -0.006305033155777784, -0.009626955475330877, 0.024522257815088908, -0.011428571428571453, 0.030646744465045287, 0.035873015873015876, 0.030442333231177895, 0.0286507385744027, -0.08905165767154981, 0.04041472704189598, -0.06985967053081152, -0.06942166830654853, 0.007283834586466051, -0.0068

Row,stock,b0,b1,b2,b3,b4,b5,μ
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Amazon,0.00141525,0.0937805,-0.100002,-0.0800912,0.263179,-0.188408,0.0031763
2,American_Express,0.00294212,-0.176957,-0.0507746,0.238879,0.0982535,-0.0440592,0.0307976
3,Apple,-0.000978696,-0.098789,-0.000987782,0.260684,-0.247344,0.00841229,0.044612
4,Boeing,-0.00197361,-0.254758,-0.410695,-0.102175,0.134665,-0.0624203,0.0667526
5,Coca_Cola,0.00308611,0.118958,0.0993294,0.0692586,-0.0891389,0.0761207,0.00789937
6,Goldman_Sachs,0.00354639,0.0234379,-0.146093,0.27689,0.0441058,-0.147334,0.0309844
7,JPMorgan_Chase,0.00881022,-0.0809667,-0.265347,0.270727,-0.0844002,-0.405715,0.0667482
8,Johnson_Johnson,0.000503491,0.244037,0.0365928,0.0747965,-0.0831539,0.182775,-0.010924
9,Microsoft,-0.00335244,-0.103942,-0.17504,-0.150991,0.0416056,-0.144069,0.00720825


In [18]:
const M = 5                # number of different models
W = zeros(n, M)            # each column i will hold the w from model i

10×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

## Model 5

In [ ]:
# # ─── SET UP & SOLVE OPTIMIZATION ──────────────────────────────────────────────

# γ      = fill(100.0, n)
# L      = liquidity
# λ      = 0.99
# w_prev = fill(1/n, n)

# model = Model(Gurobi.Optimizer)
# @variable(model, w[1:n])
# @variable(model, z[1:n] >= 0)

# @constraint(model, sum(w) == 1)
# @constraint(model, w .>= 0)
# @constraint(model, w .<= 1)

# # |w[i] - w_prev[i]| <= z[i]
# @constraint(model, [i=1:n], z[i] ≥ w[i] - w_prev[i])
# @constraint(model, [i=1:n], z[i] ≥ -(w[i] - w_prev[i]))

# @objective(model, Min,
#     λ * w' * Σ * w
#   - (1 - λ) * (dot(w, μ)+ λ * sum(z[i] / L[i] for i in 1:n ))
# )

# optimize!(model)

# println("\nObjective value: ", objective_value(model))
# println("Optimal portfolio weights:")
# for i in 1:n
#     println("  ", lpad(names[i], 10), " → ", round(value(w[i]), digits=4))
# end
# ─── SET UP & SOLVE OPTIMIZATION (MIQP with Fixed Costs) ─────────────────────
# The coefficients of the piecewise linear transaction cost function
using LinearAlgebra            # for cholesky
using Distributions            # only if you want to compute z_alpha dynamically
using JuMP
using Gurobi

# --- data & parameters as before ---
# Σ ∈ ℝ^{n×n}, μ ∈ ℝ^n, L[i]=liquidity[i], etc.
γ      = 1000_000
λ      = 0.99
w_prev = fill(1/n, n)
ϕ_trade = 0.002
ϕ_hold  = 0.001

# VaR parameters
α       = 0.95
z_alpha = quantile(Normal(), 0.95) 
τ       = - 0.02                  # your loss‐threshold

# Cholesky factor of Σ
Lmat = cholesky(Σ).L

model = Model(Gurobi.Optimizer)

# --- variables ---
@variable(model, w[1:n] ≥ 0, upper_bound = 1)
@variable(model, z[1:n] ≥ 0)
@variable(model, y[1:n], Bin)
@variable(model, h[1:n], Bin)
@variable(model, cvar[1:n] ≥ 0)
@variable(model, variance ≥ 0)

# NEW for VaR:
@variable(model, σ_p ≥ 0)                       # portfolio std‐dev
@expression(model, Lw[i=1:n], sum(Lmat[i,j]*w[j] for j=1:n))

# --- constraints ---
@constraint(model, sum(w) == 1)

# transaction‐cost linearization
@constraint(model, [i=1:n], z[i] ≥ w[i] - w_prev[i])
@constraint(model, [i=1:n], z[i] ≥ -(w[i] - w_prev[i]))
@constraint(model, [i=1:n], z[i] ≤ y[i])
@constraint(model, [i=1:n], w[i] ≤ h[i])

# piecewise‐linear cost
for i in 1:n, k in 1:100
    @constraint(model, cvar[i] ≥ a[k] * z[i] + b[k])
end

# variance
@constraint(model, variance == dot(w, Σ*w))

# Second‐order‐cone for σ_p ≥ sqrt(w' Σ w)
@constraint(model, [σ_p; Lw...] in SecondOrderCone())

# VaR constraint: -μᵀw + z_α σ_p ≤ τ
@constraint(model, dot(μ, w) - z_alpha * σ_p >= τ)

# --- objective ---
@objective(model, Min,
    λ * variance
  - (1 - λ) * (dot(μ, w) - γ * sum(cvar[i] / L[i] for i in 1:n)
                        - ϕ_trade * sum(y)
                        - ϕ_hold  * sum(h))
)

optimize!(model)

println("Objective value: ", objective_value(model))
println("Optimal weights:")
for i in 1:n
    println("  ", names[i], ": ", round(value(w[i]), digits=4))
end

w_opt = value.(w)       # this is a Vector{Float64} of length n
W[:, 5] = w_opt;



Set parameter Username
Set parameter LicenseID to value 2650745
Academic license - for non-commercial use only - expires 2026-04-12
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 1053 rows, 63 columns and 2168 nonzeros
Model fingerprint: 0x62c42927
Model has 2 quadratic constraints
Variable types: 43 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [5e-05, 1e+01]
  QMatrix range    [5e-05, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e-05, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-03, 5e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 1263 rows, 127 columns, 3640 nonzeros
Presolved model has 10 quadratic constraint(s)
Presolved model has 55 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 107 continuous, 20 integer 

Caculate the realized return in the last month using our porfolio strategy.

In [7]:
# actual_returns = [
#     (185.01 - 184.87) / 184.87,
#     (275.37 - 251.13) / 251.13,
#     (198.51 - 198.15) / 198.15,
#     (185.96 - 156.84) / 156.84,
#     (71.72 - 71.43) / 71.43,
#     (549.36 - )
# ]

# the two dates of interest
d1 = Date("Apr 7, 2025",  dateformat"u d, Y")
d2 = Date("May 5, 2025",  dateformat"u d, Y")

# container for the actual returns
actual_returns = Float64[]

for file in csv_files
    df = CSV.read(file, DataFrame)
    df.Date = Date.(df.Date, dateformat"u d, Y")
    sort!(df, :Date)  # make sure it’s sorted

    # look up the two adj_close prices
    # `findfirst` will give the row index where the date matches
    i1 = findfirst(==(d1), df.Date)
    i2 = findfirst(==(d2), df.Date)

    if isnothing(i1) || isnothing(i2)
        error("Date $d1 or $d2 not found in $(basename(file))")
    end

    p1 = df.Adj_Close[i1]
    p2 = df.Adj_Close[i2]

    push!(actual_returns, (p2 - p1) / p1)
end

# assemble into a nice DataFrame
actual = DataFrame(
    name   = names,
    ret20d = actual_returns,   # 20‐trading‐day return
)

display(actual)

Row,name,ret20d
,String,Float64
1,Amazon,0.000757289
2,American_Express,0.0965237
3,Apple,0.00181681
4,Boeing,0.185667
5,Coca_Cola,0.00405992
6,Goldman_Sachs,0.111075
7,JPMorgan_Chase,0.0552498
8,Johnson_Johnson,0.0180584
9,Microsoft,0.115485


In [17]:
# — after optimize!(model) and after building `actual_returns` —

# 1) extract optimal weights
w_opt = value.(w)

# 2) compute portfolio realized return
realized_ret = dot(w_opt, actual_returns)

println("\nRealized portfolio return over Apr 7→May 5, 2025: ",
        round(realized_ret*100, digits=2), "%")



Realized portfolio return over Apr 7→May 5, 2025: 4.72%
